# B사 소재코드 찾기

- 프로그램 개요
    1. B사와 유사한 소재가 포함된 모든 표준정보를 정리한다.
    1. B사 소재코드에는 조건부서식을 적용하여 셀 강조
    1. 소재 리스트가 변경되었을 경우, 변경된 리스트만 따로 엑셀파일로 제공하면 작동한다.

## 학생복 시스템 접속해서 표준정보 불러오기

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import binascii   # 한글 변환에 필요한 라이브러리
from timeit import default_timer as timer
import sys

### 수행시간측정 시작

In [2]:
start_lab = timer()

### DB 접속

In [3]:
sys.path.append('/settings')
import config

def connect_db(sid):
    if sid != config.COMPANY_DB_CONFIG['sid']:
        raise ValueError("DB 를 찾을 수 없습니다.")
    
    conn = create_engine(
        "oracle+cx_oracle://{user}:{password}@{host}:{port}/{sid}?encoding=UTF-8&nencoding=UTF-8".format(
            user=config.COMPANY_DB_CONFIG['user'],
            password=config.COMPANY_DB_CONFIG['password'],
            host=config.COMPANY_DB_CONFIG['host'],
            port=config.COMPANY_DB_CONFIG['port'],
            sid=config.COMPANY_DB_CONFIG['sid']
        )
    )
    
    return conn

engine = connect_db('IVY')

### SQL 본문

In [4]:
sql1 = f'''
SELECT z.bkjk_year,
       z.bkjk_season,
       sch_tkyk,
       Rawtohex(utl_raw.Cast_to_raw(tkyk_name)) tkyk_name,
       sch_agen,
       Rawtohex(utl_raw.Cast_to_raw(agen_name)) agen_name,
       z.bkjk_school,
       Rawtohex(utl_raw.Cast_to_raw(sch_name))  sch_name,
       decode(substr(sch_code,2,1), 'A', 'M', 'C', 'M', 'E', 'M', 'G', 'M', 'F') gubun,
       z.bkjk_bokjong,
       z.bkjk_sojae,
       z.bkjk_bsojae1,
       z.bkjk_bsojae2,
       z.bkjk_bsojae3,
       z.bkjk_bsojae4,
       z.bkjk_bsojae5
FROM   (SELECT bkjk_year,
               bkjk_season,
               bkjk_school,
               bkjk_bokjong,
               bkjk_sojae,
               bkjk_bsojae1,
               bkjk_bsojae2,
               bkjk_bsojae3,
               bkjk_bsojae4,
               bkjk_bsojae5
        FROM   i_stand_bkjk_t
        WHERE  bkjk_year = '23'
               AND bkjk_season = 'F'
        UNION ALL
        SELECT bkjk_year,
               bkjk_season,
               bkjk_school,
               bkjk_bokjong,
               bkjk_sojae,
               bkjk_bsojae1,
               bkjk_bsojae2,
               bkjk_bsojae3,
               bkjk_bsojae4,
               bkjk_bsojae5
        FROM   i_stand_bkjk_t
        WHERE  bkjk_year = '22'
               AND bkjk_season = 'S') z,
       i_sch_t,
       i_tkyk_t,
       i_agen_t
WHERE  z.bkjk_school = sch_code(+)
       AND sch_tkyk = tkyk_code(+)
       AND sch_agen = agen_code(+)
'''

### 한글변환 함수

In [5]:
# # US7ASCII -> CP949(완성형한글) 로 변환
def us7ascii_to_cp949(df):
    for index, byte_data in enumerate(df):
        if byte_data == None: # null 값이면 패스. 안하면 변환 에러난다.
            continue
        byte_data = binascii.unhexlify(df[index])  # 16진수 문자열 hexstr로 표현된 바이너리 데이터를 반환. 역함수는 b2a_hex()
        df[index] = byte_data.decode("cp949")  # 바이트 변환값 -> cp949(완성형 한글) 로 변환
    return df

### 표준정보 쿼리 함수

In [6]:
def sql_query(oracleSql):
    df = pd.read_sql_query(oracleSql, engine)

    if 'tkyk_name' in df.columns: # 해당컬럼이 없어도 에러없이 처리
        df_temp = df['tkyk_name'].copy()
        df['tkyk_name'] = us7ascii_to_cp949(df_temp)

    if 'agen_name' in df.columns:
        df_temp = df['agen_name'].copy()
        df['agen_name'] = us7ascii_to_cp949(df_temp)

    if 'sch_name' in df.columns:
        df_temp = df['sch_name'].copy()
        df['sch_name'] = us7ascii_to_cp949(df_temp)
    
    return df

In [7]:
df = sql_query(sql1)
df

,bkjk_year,bkjk_season,sch_tkyk,tkyk_name,sch_agen,agen_name,bkjk_school,sch_name,gubun,bkjk_bokjong,bkjk_sojae,bkjk_bsojae1,bkjk_bsojae2,bkjk_bsojae3,bkjk_bsojae4,bkjk_bsojae5
0,23,F,R,부산상권,VP131,금정점,PE002,동래초등,M,J,ACS0999,ZWP0209,None,None,None,None
1,23,F,R,부산상권,VP131,금정점,PE002,동래초등,M,K,BPV0991,BPV0175,None,None,None,None
2,23,F,R,부산상권,VP131,금정점,PE002,동래초등,M,G,BPV0991,BPV0175,None,None,None,None
3,23,F,R,부산상권,VP131,금정점,PF002,동래초등(여),F,J,ACS0999,ZWP0209,None,None,None,None
4,23,F,R,부산상권,VR150,진주점,RB058,진주여중,F,S,UGD0603,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102618,22,S,H,중부상권,None,None,AA714,푸른솔중,M,D,None,None,None,None,None,None
102619,22,S,H,중부상권,None,None,AA714,푸른솔중,M,Z,None,None,None,None,None,None
102620,22,S,H,중부상권,VB132,가평점,AA755,가평중,M,D,None,None,None,None,None,None
102621,22,S,C,서울상권,VA338,부천점,AA279,부흥중남(부천),M,Z,CPU0048,None,None,None,None,None


### B사 대상소재 불러오기

- 현재 시스템 상에서 확정되지 않고 파일로만 있음

In [8]:
df_soje = pd.read_excel('bur_sojae.xlsx', sheet_name='Sheet1')
df_soje

,코드,소재
0,APR0760,타소재
1,APR0765,타소재
2,BPR0703,타소재
3,BPR0764,타소재
4,BPR0810,타소재
...,...,...
83,ZTP1554,하소모
84,ZTP1570,하소모
85,ZTP1576,하소모
86,ZTP1583,하소모


### 데이터 처리 시작

#### 각 표준 컬럼 단위로 B사 소재 포함여부를 확인하여 데이터프레임 생성

In [9]:
df_s1 = df[df['bkjk_sojae'].isin(df_soje['코드'])]
df_b1 = df[df['bkjk_bsojae1'].isin(df_soje['코드'])]
df_b2 = df[df['bkjk_bsojae2'].isin(df_soje['코드'])]
df_b3 = df[df['bkjk_bsojae3'].isin(df_soje['코드'])]
df_b4 = df[df['bkjk_bsojae4'].isin(df_soje['코드'])]
df_b5 = df[df['bkjk_bsojae5'].isin(df_soje['코드'])]

#### 저장한 6개 데이터프레임 합치기

In [10]:
df_total = pd.concat([df_s1,
                      df_b1,
                      df_b2,
                      df_b3,
                      df_b4,
                      df_b5                 
                     ])
df_total

,bkjk_year,bkjk_season,sch_tkyk,tkyk_name,sch_agen,agen_name,bkjk_school,sch_name,gubun,bkjk_bokjong,bkjk_sojae,bkjk_bsojae1,bkjk_bsojae2,bkjk_bsojae3,bkjk_bsojae4,bkjk_bsojae5
130,23,F,L,광주상권,VN124,곡성점,KB073,담양여중,F,S,ZWP1608,None,None,None,None,None
158,23,F,L,광주상권,VN124,곡성점,KB073,담양여중,F,R,ZWP1608,None,None,None,None,None
197,23,F,D,대전상권,VH027,아산점,HD033,한올여고(WHITE),F,S,ZWP1354,None,None,None,None,None
205,23,F,D,대전상권,VH027,아산점,HD033,한올여고(WHITE),F,R,ZWP1354,None,None,None,None,None
959,23,F,H,중부상권,VA341,화성점,AB533,동화중여(화성),F,R,ZWP1267,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79427,22,S,C,서울상권,VS470,가양점,SD310,세현고여,F,N,HPT0501,ZTS0450,APR0195,SQQ0354,None,None
82604,22,S,C,서울상권,VS039,양천점,SC038,백암고남,M,Y,UPR0103,ZTP0579,SQQ0318,SQQ0316,None,None
85034,22,S,H,중부상권,VH000,[미취급],AA255,양곡중,M,Y,UPR0103,ZTS0068,SQQ0344,SQQ0316,SQQ0318,None
86719,22,S,A,본사,VA000,[미취급],DD036,기획가디건(진곤색),F,Y,UPR0103,SQQ0105,ZTP0648,SQQ0318,SQQ0316,None


In [11]:
df.columns

Index(['bkjk_year', 'bkjk_season', 'sch_tkyk', 'tkyk_name', 'sch_agen',
       'agen_name', 'bkjk_school', 'sch_name', 'gubun', 'bkjk_bokjong',
       'bkjk_sojae', 'bkjk_bsojae1', 'bkjk_bsojae2', 'bkjk_bsojae3',
       'bkjk_bsojae4', 'bkjk_bsojae5'],
      dtype='object')

#### 중복제거 후 최종결과물

In [12]:
df_total = df_total.drop_duplicates()
df_total.columns = ['년도',
                    '시즌',
                    '상권코드',
                    '상권',
                    '대리점코드',
                    '대리점명',
                    '학교코드',
                    '학교명',
                    '남녀구분',
                    '복종',
                    '소재',
                    '배색1',
                    '배색2',
                    '배색3',
                    '배색4',
                    '배색5'
                   ]
df_total

,년도,시즌,상권코드,상권,대리점코드,대리점명,학교코드,학교명,남녀구분,복종,소재,배색1,배색2,배색3,배색4,배색5
130,23,F,L,광주상권,VN124,곡성점,KB073,담양여중,F,S,ZWP1608,None,None,None,None,None
158,23,F,L,광주상권,VN124,곡성점,KB073,담양여중,F,R,ZWP1608,None,None,None,None,None
197,23,F,D,대전상권,VH027,아산점,HD033,한올여고(WHITE),F,S,ZWP1354,None,None,None,None,None
205,23,F,D,대전상권,VH027,아산점,HD033,한올여고(WHITE),F,R,ZWP1354,None,None,None,None,None
959,23,F,H,중부상권,VA341,화성점,AB533,동화중여(화성),F,R,ZWP1267,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79427,22,S,C,서울상권,VS470,가양점,SD310,세현고여,F,N,HPT0501,ZTS0450,APR0195,SQQ0354,None,None
82604,22,S,C,서울상권,VS039,양천점,SC038,백암고남,M,Y,UPR0103,ZTP0579,SQQ0318,SQQ0316,None,None
85034,22,S,H,중부상권,VH000,[미취급],AA255,양곡중,M,Y,UPR0103,ZTS0068,SQQ0344,SQQ0316,SQQ0318,None
86719,22,S,A,본사,VA000,[미취급],DD036,기획가디건(진곤색),F,Y,UPR0103,SQQ0105,ZTP0648,SQQ0318,SQQ0316,None


#### 데이터프레임 정렬 및 인덱스 리셋

In [13]:
# 기본 정렬 방식은 오름차순(ascending)입니다.
# 내림차순으로 정렬하고 싶다면 ascending 옵션을 False로 설정하면 됩니다.
df_total = df_total.sort_values(['년도', '시즌', '상권코드', '대리점코드', '학교코드', '복종']).reset_index(drop=True)

#### 집계

In [14]:
df_group = df_total.copy()
df_group['집계'] = 1

df_gr_tkyk = df_group.groupby(['상권'])[['집계']].agg(sum)
df_gr_agent = df_group.groupby(['상권', '대리점명'])[['집계']].agg(sum)
df_gr_season = df_group.groupby(['시즌'])[['집계']].agg(sum)
df_gr_bok = df_group.groupby(['시즌', '복종'])[['집계']].agg(sum)

df_gr_tkyk

,집계
상권,
광주상권,202
대구상권,146
대전상권,212
본사,21
부산상권,114
서울상권,428
유니폼본사,1
중부상권,420


#### B사소재 series를 list로 변환

In [15]:
bur_sojae = df_soje['코드'].to_list()

#### 조건부서식 (셀 스타일 배경색 추가, 텍스트 굵게 적용)

In [16]:
def draw_color_in_bursojae(value, color, font_weight):
    if str(value) in bur_sojae:
        fill1 = color
        fill2 = font_weight
    else:
        fill1 = "inherit"
        fill2 = "inherit"

    return f"background-color: {fill1};font-weight: {fill2}"

df_total = df_total.style.applymap(draw_color_in_bursojae, color='#FFC000', font_weight='bold')
df_total

### 자료 작성기준일 입력

In [17]:
from datetime import datetime

In [18]:
today = datetime.today().strftime("%Y%m%d")
today

'20220617'

### 파일저장 및 종료

In [19]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(f'{today}_B사소재표준정보.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df_total.to_excel(writer, sheet_name='표준', index=False)
df_soje.to_excel(writer, sheet_name='B사소재리스트', index=False)

df_gr_tkyk.to_excel(writer, sheet_name='상권별_집계')
df_gr_agent.to_excel(writer, sheet_name='대리점별_집계')
df_gr_season.to_excel(writer, sheet_name='시즌별_집계')
df_gr_bok.to_excel(writer, sheet_name='복종별_집계')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

### 수행시간측정 종료

In [20]:
end_lab = timer()

In [21]:
print(f'총 수행시간 : {end_lab-start_lab} 초')

총 수행시간 : 13.064146399999998 초
